In [4]:
from pyspark import SparkContext                                                                                        
from pyspark.sql import SparkSession                                                                                    
from pyspark.streaming import StreamingContext                                                                          
from pyspark.streaming.kafka import KafkaUtils    
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import pandas as pd
import json
import time
import yaml
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

In [2]:
ss = SparkSession.Builder() \
     .appName("SparkSpeedStreamingKafka") \
     .master("spark://speed-processing-spark-master:7077") \
     .config("spark.jars", "./spark-sql-kafka-0-10_2.11-2.4.1.jar,./kafka-clients-2.0.0.jar") \
     .getOrCreate()


In [3]:
line = ss \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka-broker-1:9093,kafka-broker-2:9093,kafka-broker-3:9093") \
  .option("subscribe", "trips") \
  .load()


In [10]:
def transform(s: str):
    """
    s: '{"ID": 114, "ArrivalTime": "1419322880", "BusinessLeisure": "nan", "CabinCategory": "40", "CreationDate": "2456957", "CurrencyCode": "nan", "DepartureTime": "1419611264", "Destination": "HKG", "OfficeIdCountry": "TH", "Origin": "BKK", "TotalAmount": "nan", "nPAX": "1"}'
    """
    return str(len(s))

In [11]:
df = line.select('value')
udf_transform = udf(transform)
df = df.withColumn("value", udf_transform("value"))

In [ ]:
query = df \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka-broker-1:9093,kafka-broker-2:9093,kafka-broker-3:9093") \
    .option("topic", "real-time-statistic") \
    .option("checkpointLocation", "/tmp/checkpoint") \
    .start()

query.awaitTermination()